# Strands Graph with Explicit invocation_state
Agent-as-Tool 패턴으로 invocation_state 명시적 전달

In [10]:
from strands import Agent, tool, ToolContext
from strands.multiagent import GraphBuilder
from strands.models import BedrockModel

bedrock_model = BedrockModel(
    model_id="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
    temperature=0.3
)

## 1. 기본 데이터 수집 Tools

In [11]:
@tool(context=True)
def collect_entity(entity: str, entity_type: str, tool_context: ToolContext) -> str:
    entities = tool_context.invocation_state.get("entities", [])
    entities.append({"name": entity, "type": entity_type})
    tool_context.invocation_state["entities"] = entities
    print(f"✓ Entity: {entity} ({entity_type})")
    return f"Collected {entity}"

@tool(context=True)
def collect_relation(source: str, target: str, relation: str, tool_context: ToolContext) -> str:
    relations = tool_context.invocation_state.get("relations", [])
    relations.append({"source": source, "target": target, "relation": relation})
    tool_context.invocation_state["relations"] = relations
    print(f"✓ Relation: {source} --[{relation}]--> {target}")
    return f"Collected relation"

## 2. Agent-as-Tool (invocation_state 명시적 전달)

In [12]:
@tool(context=True)
def entity_extractor_tool(text: str, tool_context: ToolContext) -> str:
    """엔티티 추출 Agent (invocation_state 명시적 전달)"""
    agent = Agent(
        name="entity_extractor",
        model=bedrock_model,
        system_prompt="Extract all entities from text and use collect_entity tool.",
        tools=[collect_entity]
    )
    # invocation_state 명시적 전달
    result = agent(text, invocation_state=tool_context.invocation_state)
    return str(result)

@tool(context=True)
def relation_extractor_tool(text: str, tool_context: ToolContext) -> str:
    """관계 추출 Agent (invocation_state 명시적 전달)"""
    agent = Agent(
        name="relation_extractor",
        model=bedrock_model,
        system_prompt="Extract relationships between entities and use collect_relation tool.",
        tools=[collect_relation]
    )
    # invocation_state 명시적 전달
    result = agent(text, invocation_state=tool_context.invocation_state)
    return str(result)

@tool(context=True)
def summarizer_tool(text: str, tool_context: ToolContext) -> str:
    """요약 Agent (invocation_state 조회)"""
    entities = tool_context.invocation_state.get("entities", [])
    relations = tool_context.invocation_state.get("relations", [])
    
    summary = f"Extracted {len(entities)} entities and {len(relations)} relations."
    
    agent = Agent(
        name="summarizer",
        model=bedrock_model,
        system_prompt=f"Summarize the graph: {summary}",
        tools=[]
    )
    result = agent(text, invocation_state=tool_context.invocation_state)
    return str(result)

## 3. Wrapper Agents for Graph Nodes

In [13]:
# Graph 노드용 Wrapper Agent
entity_node = Agent(
    name="entity_node",
    model=bedrock_model,
    system_prompt="Extract entities using entity_extractor_tool.",
    tools=[entity_extractor_tool]
)

relation_node = Agent(
    name="relation_node",
    model=bedrock_model,
    system_prompt="Extract relations using relation_extractor_tool.",
    tools=[relation_extractor_tool]
)

summary_node = Agent(
    name="summary_node",
    model=bedrock_model,
    system_prompt="Summarize using summarizer_tool.",
    tools=[summarizer_tool]
)

## 4. Build Graph

In [14]:
builder = GraphBuilder()

builder.add_node(entity_node, "extract_entities")
builder.add_node(relation_node, "extract_relations")
builder.add_node(summary_node, "summarize")

builder.add_edge("extract_entities", "extract_relations")
builder.add_edge("extract_relations", "summarize")

builder.set_entry_point("extract_entities")

graph = builder.build()

Graph without execution limits may run indefinitely if cycles exist


## 5. Execute Graph

In [15]:
result = graph(
    "AWS Lambda integrates with Amazon S3 and DynamoDB. S3 stores data that Lambda processes.",
    invocation_state={"entities": [], "relations": []}
)

print("\n=== Graph Result ===")
print(f"Status: {result.status}")
print(f"Execution order: {[node.node_id for node in result.execution_order]}")
print(f"\nEntities: {result.invocation_state.get('entities')}")
print(f"Relations: {result.invocation_state.get('relations')}")

I'll extract the entities from your text using the entity extraction tool.
Tool #1: entity_extractor_tool
I'll extract all the entities from the text and categorize them appropriately.
Tool #1: collect_entity

Tool #2: collect_entity

Tool #3: collect_entity

Tool #4: collect_entity

Tool #5: collect_entity
✓ Entity: AWS Lambda (service)
✓ Entity: Amazon S3 (service)
✓ Entity: DynamoDB (service)
✓ Entity: S3 (service)
✓ Entity: Lambda (service)
I've successfully extracted and collected all the entities from the text:

1. **AWS Lambda** - AWS serverless compute service
2. **Amazon S3** - AWS object storage service  
3. **DynamoDB** - AWS NoSQL database service
4. **S3** - Short reference to Amazon S3
5. **Lambda** - Short reference to AWS Lambda

All entities have been categorized as "service" type since they represent AWS cloud services mentioned in the context of their integration and data processing capabilities.The entity extraction has identified the key AWS services mentioned in y

AttributeError: 'GraphResult' object has no attribute 'invocation_state'

## invocation_state 전달 흐름

```
Graph (invocation_state 초기화)
  ↓
entity_node (Wrapper Agent)
  ↓ tool_context.invocation_state
entity_extractor_tool (Agent-as-Tool)
  ↓ invocation_state=tool_context.invocation_state
entity_extractor (Inner Agent)
  ↓
collect_entity (Tool) → invocation_state["entities"] 수정
  ↓
relation_node (Wrapper Agent)
  ↓ tool_context.invocation_state
relation_extractor_tool (Agent-as-Tool)
  ↓ invocation_state=tool_context.invocation_state
relation_extractor (Inner Agent)
  ↓
collect_relation (Tool) → invocation_state["relations"] 수정
  ↓
summary_node (Wrapper Agent)
  ↓ tool_context.invocation_state
summarizer_tool (Agent-as-Tool) → invocation_state 조회
```